In [1]:
import pandas as pd

from funciones.divide_fecha import *
from funciones.limpia_columnas import *  
from funciones.limpia_intercambio import * 
from funciones.limpia_balance import *
from funciones.limpia_generacion import *
from funciones.limpia_demanda import *

---

### LIMPIEZA GENÉRICA 

- divide_fecha():

    Extraemos año, mes y día para cada dataframe, y lo transformamos a int. Esto hace que ahorremos unos cuantos kb de memoria.

- limpia_columnas():

    * Renombramos algunas columnas para mayor legibilidad
    * Creamos una columna "ID" juntando la fecha de extracción (sólo los números) y el índice correspondiente al dataframe importado ("Unnamed: 0"), para hacer un id individual.
    * Transformamos la columna "ID" a int (64, 32 no soporta tantos caracteres).
    * Reasignamos valores en "composite" a 1 y 0. Estos valores tienen que ver con la composición de la energía, consideramos interesante mantener la columna por ahora.
    * Eliminamos las columnas ["magnitude", "Unnamed: 0", "color", "descripcion"]

--- 

### BALANCE:  limpia_balance()

Los datos vienen con muchos valores que representan totales de agrupaciones, así que:

- Eliminamos las filas en columna "titulo" que representan totales: 
    * "Generación renovable"
    * "Generación no renovable"
    * "Saldo almacenamiento"
    * "Demanda en b.c."

- Definimos titulos_renovables, titulos_no_renovables y almacenamiento para categorizar los tipos de energía. Esto nos ayudará a reagruparlas para operar sobre ellas y posteriormente graficarlas. Además, esta reagrupación rellena NaNs que se corresponden con las categorías de la columna "titulo"

- La fila "last-update" tiene algunos valores inconsistentes así que eliminamos la fila, ya que tenemos fila con fecha de toma de la medida y fecha de la extraccion de a información.

--- 

### INTERCAMBIO:  limpia_intercambio()

- Reasignamos NaNs a "Enlace Península-Baleares": 
    
    Tras comparar "frontera" con "categoria" vemos que cuando "frontera" es NaN "categoria" = "Enlace Península-Baleares". 

- Eliminamos los valores de titulo "saldo", ya que representa totales que realmente podemos calcular ad hoc para estadísticas o gráficos.

--- 

### GENERACION:  limpia_generacion()

* Sustituimos en columna "tipo" "No-Renovable" por "No renovable". Estos valores hacen referencia a lo mismo, pero tenían 2 nombres diferentes.



* La columna tiene categorías importantes que realmente también existen en "titulo", pero en esta última están desglosadas, por lo que nos ofrece una información adicional que consideramos interesante. 


---

### DEMANDA

- Eliminamos la columna "tipo": esta columna está compuesta íntegramente por NaNs, así que podemos eliminarla.

- En la categoría "Perdidas de transporte" hay 2 valores duplicados en la columna "titulo" que además representan el total de la demanda, que podemos calcular directamente en df_balance_historico. Eliminamos las filas que correspondan a "titulo" = Demanda.

---

### EJECUCION LIMPIEZA

In [6]:
df_balance_historico = pd.read_csv("../data/raw/DF_BALANCE_HISTORICO_10_25.csv")
df_demanda_historico = pd.read_csv("../data/raw/DF_DEMANDA_HISTORICO_10_25.csv")
df_generacion_historico = pd.read_csv("../data/raw/DF_GENERACION_HISTORICO_10_25.csv")
df_intercambios_historico = pd.read_csv("../data/raw/DF_INTERCAMBIOS_HISTORICO_10_25.csv")

In [7]:
dataframes = [df_balance_historico, df_intercambios_historico, df_demanda_historico, df_generacion_historico]

dataframes = [divide_fecha(dataframe) for dataframe in dataframes]
dataframes = [limpia_columnas(dataframe) for dataframe in dataframes]

# Reasignamos para que los cambios en las funciones sean efectivos
df_balance_historico, df_intercambios_historico, df_demanda_historico, df_generacion_historico = dataframes

df_intercambios_historico = limpia_intercambio(df_intercambios_historico)
df_balance_historico = limpia_balance(df_balance_historico)
df_generacion_historico = limpia_generacion(df_generacion_historico)
df_demanda_historico = limpia_demanda(df_demanda_historico)

In [8]:
# Compruebo que no haya NaNs y tipos de las variables

for i in [df_balance_historico, df_intercambios_historico, df_demanda_historico, df_generacion_historico]:
    print(i.info())
    print("-"*50)

<class 'pandas.core.frame.DataFrame'>
Index: 324594 entries, 0 to 432489
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   titulo            324594 non-null  object        
 1   tipo              324594 non-null  object        
 2   composite         324594 non-null  int64         
 3   fecha_extraccion  324594 non-null  datetime64[ns]
 4   categoria         324594 non-null  object        
 5   zona              324594 non-null  object        
 6   valor_(MWh)       324594 non-null  float64       
 7   porcentaje        324594 non-null  float64       
 8   fecha             324594 non-null  datetime64[ns]
 9   año               324594 non-null  int32         
 10  mes               324594 non-null  int32         
 11  dia               324594 non-null  int32         
 12  dia_semana        324594 non-null  object        
 13  ID                324594 non-null  object        
dtypes: dateti

In [9]:
# guardamos los csv limpios

df_balance_historico.to_csv("../data/processed/DF_BALANCE_10_25_LIMPIO.csv", index=False, sep=",")
df_demanda_historico.to_csv("../data/processed/DF_DEMANDA_10_25_LIMPIO.csv", index=False, sep=",")
df_generacion_historico.to_csv("../data/processed/DF_GENERACION_10_25_LIMPIO.csv", index=False, sep=",")
df_intercambios_historico.to_csv("../data/processed/DF_INTERCAMBIOS_10_25_LIMPIO.csv", index=False, sep=",")